In [3]:
from utils import load_data
import torch
import numpy as np
import os
import pickle
import argparse
from einops import rearrange

from utils import compute_dict_mean, set_seed, detach_dict # helper functions
from policy import ACTPolicy, CNNMLPPolicy, DiffusionPolicy
import collections
from collections import deque


import time
import threading
import math
import threading


import sys
sys.path.append("./")


In [6]:
def make_policy(policy_class, policy_config):
    if policy_class == 'ACT':
        policy = ACTPolicy(policy_config)
    elif policy_class == 'CNNMLP':
        policy = CNNMLPPolicy(policy_config)
    elif policy_class == 'Diffusion':
        policy = DiffusionPolicy(policy_config)
    else:
        raise NotImplementedError
    return policy

In [7]:
task_config = {'camera_names': ['cam_high',  'cam_right_wrist']}

policy_config = {'lr':1e-5,
                         'lr_backbone': 1e-5,
                         'backbone': 'resnet18',
                         'masks':None,
                         'weight_decay': 1e-4,
                         'dilation': None,
                         'position_embedding': "sine",
                         'loss_function':"l1",
                         'chunk_size': 4,     # 查询
                         'camera_names':['cam_high',  'cam_right_wrist'],
                         'use_depth_image': False,
                         'use_robot_base': False,
                         'observation_horizon': 1,
                         'action_horizon': 8,
                         'num_inference_timesteps': 10,
                         'ema_power': 0.75
                         }

In [9]:
policy = make_policy("Diffusion", policy_config)

/home/jyxc/mambaforge/envs/aloha/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jyxc/mambaforge/envs/aloha/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


number of parameters: 6.715418e+07
number of parameters: 89.55M


In [11]:
train_dataloader, val_dataloader, stats, _ = load_data("/home/jyxc/GitHub/act-plus-plus/data/single_arm_bottle2-post", 10, 0,
                                                           False, False, ['cam_high',  'cam_right_wrist'],
                                                          1, 1)


Data from: /home/jyxc/GitHub/act-plus-plus/data/single_arm_bottle2-post

qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)
qpos shape: (200, 7)


In [19]:
state_dict = torch.load("/home/jyxc/GitHub/cobot_magic/aloha-devel/diffusion/model_save_observation_horizon1_observation_horizon8/policy_best.ckpt")
new_state_dict = {}
for key, value in state_dict.items():
    if key in ["model.is_pad_head.weight", "model.is_pad_head.bias"]:
        continue
    if key in ["model.input_proj_next_action.weight", "model.input_proj_next_action.bias"]:
        continue
    new_state_dict[key] = value
loading_status = policy.deserialize(new_state_dict)

Loaded model
Loaded EMA


In [20]:
def forward_pass(policy_config, data, policy):
    #print("policy config:", policy_config)
    image_data, image_depth_data, qpos_data, action_data, action_is_pad = data
    # pdb.set_trace()
    # if policy_config['select_arm']:
    #     qpos_data = qpos_data[:, 7:]
    #     action_data = action_data[:, :, 7:]
        
    (image_data, qpos_data, action_data, action_is_pad) = (image_data.cuda(), qpos_data.cuda(),
                                                           action_data.cuda(), action_is_pad.cuda())
    if policy_config['use_depth_image']:
        image_depth_data = image_depth_data.cuda()
    else:
        image_depth_data = None
    return policy(image_data, image_depth_data, qpos_data, action_data, action_is_pad)



In [24]:
image_data, image_depth_data, qpos_data, action_data, action_is_pad = data

In [35]:
(image_data[0][0].numpy()*255).astype(np.uint8).shape

(3, 480, 640)

In [34]:
import PIL
PIL.Image.fromarray((image_data[0][0].numpy()*255).astype(np.uint8))

TypeError: Cannot handle this data type: (1, 1, 640), |u1

In [25]:
for i in data:
    print(i.size())

torch.Size([1, 2, 3, 480, 640])
torch.Size([1, 1])
torch.Size([1, 7])
torch.Size([1, 200, 7])
torch.Size([1, 200])


In [21]:


for batch_idx, data in enumerate(val_dataloader):
                
                forward_dict, result = forward_pass(policy_config, data, policy)
                break

In [22]:
forward_dict

{'l2_loss': tensor(0.0096, device='cuda:0', grad_fn=<MeanBackward0>),
 'loss': tensor(0.0096, device='cuda:0', grad_fn=<MeanBackward0>)}

In [16]:
result[0].shape

torch.Size([1, 200, 7])